In [1]:
import tensorflow
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow.keras import layers
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Flatten, Dropout, MaxPooling2D
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.callbacks import EarlyStopping

In [2]:
IMG_SIZE = 180

In [3]:
datagen_train = ImageDataGenerator(rescale=1./255)
datagen_test = ImageDataGenerator(rescale=1./255)


In [4]:
train_set = datagen_train.flow_from_directory("../dataset/ttte/train", target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

Found 200 images belonging to 8 classes.


In [5]:
test_set = datagen_test.flow_from_directory("../dataset/ttte/validation", target_size=(IMG_SIZE, IMG_SIZE), class_mode='categorical')

Found 40 images belonging to 8 classes.


In [6]:
train_set.class_indices

{'Edward': 0,
 'Emily': 1,
 'Gordon': 2,
 'Henry': 3,
 'James': 4,
 'Percy': 5,
 'Thomas': 6,
 'Toby': 7}

In [7]:
test_set.class_indices

{'Edward': 0,
 'Emily': 1,
 'Gordon': 2,
 'Henry': 3,
 'James': 4,
 'Percy': 5,
 'Thomas': 6,
 'Toby': 7}

In [8]:
STEP_SIZE_TRAIN = train_set.samples // train_set.batch_size
STEP_SIZE_TRAIN

6

In [9]:
STEP_SIZE_TEST = test_set.samples // test_set.batch_size
STEP_SIZE_TEST

1

In [10]:
data_augmentation = Sequential(
    [

    layers.RandomFlip("horizontal"),
    layers.RandomRotation(0.1),
    layers.RandomZoom(0.1)
    ]
)


In [11]:
model = Sequential([
    layers.Input((IMG_SIZE, IMG_SIZE, 3)),
    data_augmentation,
    layers.Rescaling(1./255),
    layers.Conv2D(8,(3,3), padding='same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(16,(3,3), padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32,(3,3), padding = 'same', activation = 'relu'),
    layers.MaxPooling2D(),
    layers.Dropout(0.3),
    layers.Flatten(),
    layers.Dense(64, activation = 'relu'),
    layers.Dense(len(train_set.class_indices), activation = 'softmax')
])
model.summary()


Model: "sequential_1"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━┓
┃ Layer (type)                    ┃ Output Shape           ┃       Param # ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━┩
│ sequential (Sequential)         │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ rescaling (Rescaling)           │ (None, 180, 180, 3)    │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d (Conv2D)                 │ (None, 180, 180, 8)    │           224 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d (MaxPooling2D)    │ (None, 90, 90, 8)      │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_1 (Conv2D)               │ (None, 90, 90, 16)     │         1,168 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_1 (MaxPooling2D)  │ (None, 45, 45, 16)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ conv2d_2 (Conv2D)               │ (None, 45, 45, 32)     │         4,640 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ max_pooling2d_2 (MaxPooling2D)  │ (None, 22, 22, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dropout (Dropout)               │ (None, 22, 22, 32)     │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ flatten (Flatten)               │ (None, 15488)          │             0 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense (Dense)                   │ (None, 64)             │       991,296 │
├─────────────────────────────────┼────────────────────────┼───────────────┤
│ dense_1 (Dense)                 │ (None, 8)              │           520 │
└─────────────────────────────────┴────────────────────────┴───────────────┘

 Total params: 997,848 (3.81 MB)

 Trainable params: 997,848 (3.81 MB)

 Non-trainable params: 0 (0.00 B)

In [12]:
opt = Adam(learning_rate=0.0001)

callback = EarlyStopping(monitor='accuracy',
                         patience=20,
                         verbose=1,
                         mode="max",
            
                         baseline=0.4,
                          restore_best_weights=True
                         )


In [13]:

model.compile(
            optimizer=opt,
              loss = tensorflow.keras.losses.CategoricalCrossentropy(from_logits = True),
              metrics=['accuracy'])

history = model.fit(train_set,
                           epochs=30,
                           steps_per_epoch=STEP_SIZE_TRAIN,
                           validation_data=test_set, 
                           validation_steps=STEP_SIZE_TEST,
                           batch_size=32,
                           verbose=1,
                          #  callbacks=[callback]
                           )


Epoch 1/30


c:\Users\donof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\backend\tensorflow\nn.py:675: UserWarning: "`categorical_crossentropy` received `from_logits=True`, but the `output` argument was produced by a Softmax activation and thus does not represent logits. Was this intended?
  output, from_logits = _get_logits(


6/6 ━━━━━━━━━━━━━━━━━━━━ 5s 254ms/step - accuracy: 0.1131 - loss: 2.0796 - val_accuracy: 0.1562 - val_loss: 2.0790
Epoch 2/30
1/6 ━━━━━━━━━━━━━━━━━━━━ 0s 122ms/step - accuracy: 0.0625 - loss: 2.0798

c:\Users\donof\AppData\Local\Programs\Python\Python310\lib\site-packages\keras\src\trainers\epoch_iterator.py:116: UserWarning: Your input ran out of data; interrupting training. Make sure that your dataset or generator can generate at least `steps_per_epoch * epochs` batches. You may need to use the `.repeat()` function when building your dataset.
  self._interrupted_warning()


6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 42ms/step - accuracy: 0.0625 - loss: 2.0798 - val_accuracy: 0.1250 - val_loss: 2.0792
Epoch 3/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 160ms/step - accuracy: 0.1726 - loss: 2.0793 - val_accuracy: 0.1562 - val_loss: 2.0790
Epoch 4/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 53ms/step - accuracy: 0.0625 - loss: 2.0802 - val_accuracy: 0.0938 - val_loss: 2.0790
Epoch 5/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 215ms/step - accuracy: 0.1429 - loss: 2.0791 - val_accuracy: 0.1562 - val_loss: 2.0791
Epoch 6/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.0000e+00 - loss: 2.0804 - val_accuracy: 0.1562 - val_loss: 2.0786
Epoch 7/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 193ms/step - accuracy: 0.1250 - loss: 2.0790 - val_accuracy: 0.1562 - val_loss: 2.0788
Epoch 8/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 0s 59ms/step - accuracy: 0.1562 - loss: 2.0790 - val_accuracy: 0.1562 - val_loss: 2.0788
Epoch 9/30
6/6 ━━━━━━━━━━━━━━━━━━━━ 1s 149ms/step - accuracy: 0.1369 - loss: 2.0788 - val_accuracy: 0.0938 - val_loss: 2.0786
Epo

In [16]:
model.save("ttte_model.keras")